# Vector + Raster
In this notebook we will combine the Descartes Labs `Workflows` and `Vector` APIs to build an interactive map to **query and visualize building footprints**.  In the map below, you can draw a custom AOI anywhere in the United States, and we fetch and display the local building footprints on top of NAIP imagery.

You can run the following cells using `Shift-Enter`.

## Import packages

In [ ]:
# keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
# import packages
import descarteslabs as dl
import descarteslabs.workflows as wf
import ipyleaflet
import ipywidgets
from shapely.geometry import shape

## Specify the NAIP imagery that we'll use as our base layer.

In [ ]:
start_datetime = "2015-01-01"
end_datetime = "2019-01-01"
naip = wf.ImageCollection.from_id('usda:naip:rgbn:v1', start_datetime=start_datetime, end_datetime=end_datetime).pick_bands("red green blue")

## Specify the vector product that we will search.
We're going to display building footprints from the [Microsoft US Building Footprints database](https://github.com/microsoft/USBuildingFootprints).  This database is made available under the [Open Database License](http://opendatacommons.org/licenses/odbl/1.0/). Any rights in individual contents of the database are licensed under the [Database Contents License](http://opendatacommons.org/licenses/dbcl/1.0/).

In [ ]:
vector_id = 'a35126a241bd022c026e96ab9fe5e0ea23967d08:USBuildingFootprints'
FeatureCollection = dl.vectors.FeatureCollection(vector_id)

## Initialize our Workflows map

In [ ]:
m = wf.map
naip.max(axis='images').visualize('NAIP', scales=[(0, 1), (0, 1), (0, 1)], map=m)
m.map.center = 33.3666, -111.9696 # Phoenix
m.map.zoom = 15

## Define custom widget for interactive display of vector data on top of raster imagery

In [ ]:
draw_control = ipyleaflet.DrawControl(
    edit=False,
    remove=False,
    circlemarker={},
    polyline={},
    polygon={},
    rectangle={"shapeOptions": {
        "fillColor": "#d534eb",
        "color": "#d534eb",
        "fillOpacity": 0.2
    }}
)

# put in a popup box in the corner
output = ipywidgets.Output()
output = wf.interactive.clearable.ClearableOutput(output)
output_control = ipyleaflet.WidgetControl(
    widget=output, position="bottomright"
)
m.add_control(output_control)
# get output object back
output = output.children[0]

@output.capture()
def handle_hover(feature, **kwargs):
    output.clear_output()
    for key in sorted(feature['properties'].keys()):
        print('{}: {}'.format(key, feature['properties'][key]))

def query_features(geo_json):
    # convert to geometry
    print('Querying geometry {}'.format(geo_json['geometry']))
    geometry = shape(geo_json['geometry'])
    
    # query vector
    fs = FeatureCollection.filter(geometry=geometry).features()
    print('Downloading {} features'.format(len(fs)))
    features = [f for f in fs]
    print('Found {} features'.format(len(features)))
    
    # turn into geojson FeatureCollection
    fc = {'type': 'FeatureCollection',
          'features': [f.geojson for f in features]}
    return fc
    
    
geojsons = []
downloaded_features = []
@m.output_log.capture()
def handle_draw(other_self, action, geo_json):
    if action == 'created':
        data = query_features(geo_json)
        geolayer = ipyleaflet.GeoJSON(data=data, hover_style={'fillOpacity': 0.5})
        geolayer.on_hover(handle_hover)
        downloaded_features.extend(data['features'])
        m.add_layer(geolayer)
        # delete the drawn polygon
        other_self.clear()
    elif action == 'deleted':
        pass

draw_control.on_draw(handle_draw)
m.add_control(draw_control)

## Finally, display and interact with the map
You can draw a box anywhere in the contentinal US.  We'll filter the building footprints to those that lie within your AOI, and we'll display them on the map.

In [ ]:
m